<a href="https://www.kaggle.com/code/golammostofas/agno-framework-details?scriptVersionId=232710968" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# What is Agno ?

Agno is a lightweight library for building Reasoning Agents with memory, knowledge, tools and native multi-modal support. Use Agno to build Reasoning Agents, Multi-Modal Agents, Teams of Agents and Agentic Workflows.


## Key features
Agno is simple, fast and model agnostic. Here are some key features:

1. Lightning Fast: Agent creation is 10,000x faster than LangGraph (see [performance](https://github.com/agno-agi/agno#performance)).
2. Model Agnostic: Use any model, any provider, no lock-in.
3. Multi Modal: Native support for text, image, audio and video.
4. Multi Agent: Build teams of specialized agents.
5. Memory Management: Store agent sessions and state in a database.
6. Knowledge Stores: Use vector databases for RAG or dynamic few-shot learning.
7. Structured Outputs: Make Agents respond in a structured format.
8. Monitoring: Track agent sessions and performance in real-time on agno.com.

# What are Agents?

Agents are intelligent programs that solve problems autonomously.

Agents have memory, domain knowledge and the ability to use tools (like searching the web, querying a database, making API calls). 

Instead of a rigid binary definition, let’s think of Agents in terms of agency and autonomy.

* Level 0: Agents with no tools (basic inference tasks).
* Level 1: Agents with tools for autonomous task execution.
* Level 2: Agents with knowledge, combining memory and reasoning.
* Level 3: Teams of specialized agents collaborating on complex workflows.

## Level 0: Agents with no tools (basic inference tasks).

The simplest Agent is just an inference task, no tools, no memory, no knowledge.

In [1]:
pip install -U openai agno

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.0/599.0 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 616.7/616.7 kB 33.1 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.57.4
    Uninstalling openai-1.57.4:
      Successfully uninstalled openai-1.57.4
Note: you may need to restart the kernel to use updated packages.


In [2]:
from kaggle_secrets import UserSecretsClient 
import os
user_secrets = UserSecretsClient()
os.environ['OPENAI_API_KEY'] = user_secrets.get_secret("OPENAI_API_KEY")


In [3]:
from agno.agent import Agent
from agno.models.openai import OpenAIChat

agent = Agent(
    model=OpenAIChat(id="gpt-4o"),
    description="You are an enthusiastic news reporter with a flair for storytelling!",
    markdown=True
)
agent.print_response("Tell me about a breaking news story from New York.", stream=True)

Output()

## Level 1: Agents with tools for autonomous task execution.

This basic agent will obviously make up a story, lets give it a tool to search the web.

In [4]:
pip install -U duckduckgo-search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 43.0 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.1.7
    Uninstalling click-8.1.7:
      Successfully uninstalled click-8.1.7
Note: you may need to restart the kernel to use updated packages.


In [5]:
from agno.agent import Agent
from agno.models.openai import OpenAIChat
from agno.tools.duckduckgo import DuckDuckGoTools

agent = Agent(
    model=OpenAIChat(id="gpt-4o"),
    description="You are an enthusiastic news reporter with a flair for storytelling!",
    tools=[DuckDuckGoTools()],
    show_tool_calls=True,
    markdown=True
)
agent.print_response("Tell me about a breaking news story from New York.", stream=True)

Output()

## Level 2: Agents with knowledge, combining memory and reasoning.

Agents can store knowledge in a vector database and use it for RAG or dynamic few-shot learning.

Agno agents use Agentic RAG by default, which means they will search their knowledge base for the specific information they need to achieve their task.

In [6]:
!pip install -U lancedb tantivy pypdf
!pip install pylance

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.0/4.0 MB 82.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 16.6 MB/s eta 0:00:00
  Attempting uninstall: pypdf
    Found existing installation: pypdf 5.3.0
    Uninstalling pypdf-5.3.0:
      Successfully uninstalled pypdf-5.3.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.3/37.3 MB 46.1 MB/s eta 0:00:00


In [7]:
from agno.agent import Agent
from agno.models.openai import OpenAIChat
from agno.embedder.openai import OpenAIEmbedder
from agno.tools.duckduckgo import DuckDuckGoTools
from agno.knowledge.pdf_url import PDFUrlKnowledgeBase
from agno.vectordb.lancedb import LanceDb, SearchType

agent = Agent(
    model=OpenAIChat(id="gpt-4o"),
    description="You are a Thai cuisine expert!",
    instructions=[
        "Search your knowledge base for Thai recipes.",
        "If the question is better suited for the web, search the web to fill in gaps.",
        "Prefer the information in your knowledge base over the web results."
    ],
    knowledge=PDFUrlKnowledgeBase(
        urls=["https://agno-public.s3.amazonaws.com/recipes/ThaiRecipes.pdf"],
        vector_db=LanceDb(
            uri="tmp/lancedb",
            table_name="recipes",
            search_type=SearchType.hybrid,
            embedder=OpenAIEmbedder(id="text-embedding-3-small"),
        ),
    ),
    tools=[DuckDuckGoTools()],
    show_tool_calls=True,
    markdown=True
)

# Comment out after the knowledge base is loaded
if agent.knowledge is not None:
    agent.knowledge.load()

agent.print_response("How do I make chicken and galangal in coconut milk soup", stream=True)
agent.print_response("What is the history of Thai curry?", stream=True)

INFO Creating table: recipes

INFO Loading knowledge base

INFO Reading: https://agno-public.s3.amazonaws.com/recipes/ThaiRecipes.pdf

INFO Inserting 14 documents

INFO Added 14 documents to knowledge base

Output()

Output()

## Level 3: Teams of specialized agents collaborating on complex workflows(Multi Agent Teams).

Agents work best when they have a singular purpose, a narrow scope and a small number of tools. When the number of tools grows beyond what the language model can handle or the tools belong to different categories, use a team of agents to spread the load.

In [8]:
pip install -U yfinance

/usr/local/lib/python3.10/dist-packages/lance/__init__.py:161: UserWarning: lance is not fork-safe. If you are using multiprocessing, use spawn instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/lancedb/__init__.py:220: UserWarning: lance is not fork-safe. If you are using multiprocessing, use spawn instead.
  warnings.warn(


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.8/109.8 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.50
    Uninstalling yfinance-0.2.50:
      Successfully uninstalled yfinance-0.2.50
Note: you may need to restart the kernel to use updated packages.


In [9]:
from agno.agent import Agent
from agno.models.openai import OpenAIChat
from agno.tools.duckduckgo import DuckDuckGoTools
from agno.tools.yfinance import YFinanceTools

web_agent = Agent(
    name="Web Agent",
    role="Search the web for information",
    model=OpenAIChat(id="gpt-4o"),
    tools=[DuckDuckGoTools()],
    instructions="Always include sources",
    show_tool_calls=True,
    markdown=True,
)

finance_agent = Agent(
    name="Finance Agent",
    role="Get financial data",
    model=OpenAIChat(id="gpt-4o"),
    tools=[YFinanceTools(stock_price=True, analyst_recommendations=True, company_info=True)],
    instructions="Use tables to display data",
    show_tool_calls=True,
    markdown=True,
)

agent_team = Agent(
    team=[web_agent, finance_agent],
    model=OpenAIChat(id="gpt-4o"),
    instructions=["Always include sources", "Use tables to display data"],
    show_tool_calls=True,
    markdown=True,
)

agent_team.print_response("What's the market outlook and financial performance of AI semiconductor companies?", stream=True)

Output()

## Debugging Mode

In [10]:
from agno.agent import Agent

agent = Agent(markdown=True, debug_mode=True)
agent.print_response("Share a 2 sentence horror story")

DEBUG **************** Agent ID: 4bc83936-8caa-4482-aeae-33f166f036dc ****************

DEBUG *************** Session ID: b8ac3024-e097-416e-806b-1dd4aecaec9e ***************

DEBUG ************ Agent Run Start: 6e8449e3-ac50-41a7-b4ca-585603f26300 *************

DEBUG ---------------------------- OpenAI Response Start -----------------------------

DEBUG -------------------------------- Model: gpt-4o ---------------------------------

...

DEBUG ----------------------------- OpenAI Response End ------------------------------

DEBUG Added 2 Messages to AgentMemory

DEBUG Added AgentRun to AgentMemory

DEBUG Logging Agent Run

DEBUG ************* Agent Run End: 6e8449e3-ac50-41a7-b4ca-585603f26300 **************